In [30]:
# Set up the client

from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables from .env.

client = OpenAI(
    api_key=os.getenv("OPEN_AI_KEY")
)

assistant_id = os.getenv("ASSISTANT_ID")

server_url = os.getenv("SERVER_URL")

In [46]:
import requests

def get_github_profile(id):
    print("Getting github profile for user with id: ", id)
    response = requests.get(server_url)
    return response.text



In [32]:
# Fetch the assistant

assistant = client.beta.assistants.retrieve(
    assistant_id=assistant_id
)
assistant

Assistant(id='asst_pc2Su2UHd6IV7i3WwaYAhCqF', created_at=1712382025, description=None, file_ids=[], instructions='', metadata={}, model='gpt-4', name='github profile', object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='get_github_data', description='Retrieves data from a specific API using a GitHub ID. This function is called only if the data has not been previously retrieved.', parameters={'type': 'object', 'properties': {'github_id': {'type': 'string', 'description': 'The GitHub ID for which the data needs to be fetched'}}, 'required': ['github_id']}), type='function')])

In [33]:
# Create a thread

thread = client.beta.threads.create()
thread

Thread(id='thread_0UlN4CHI6NpzcrZnRq9Z62Dh', created_at=1712391897, metadata={}, object='thread')

In [38]:
# Prompt the model to tell us all about the data provided

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="hello plz tell how many repo i have on github. Id: ethansnow2012",
)
run

Run(id='run_s5xLDsyxoLSJY8ALRdsDhZ1w', assistant_id='asst_pc2Su2UHd6IV7i3WwaYAhCqF', cancelled_at=None, completed_at=None, created_at=1712392023, expires_at=1712392623, failed_at=None, file_ids=[], instructions='hello plz tell how many repo i have on github. Id: ethansnow2012', last_error=None, metadata={}, model='gpt-4', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_0UlN4CHI6NpzcrZnRq9Z62Dh', tools=[FunctionTool(function=FunctionDefinition(name='get_github_data', description='Retrieves data from a specific API using a GitHub ID. This function is called only if the data has not been previously retrieved.', parameters={'type': 'object', 'properties': {'github_id': {'type': 'string', 'description': 'The GitHub ID for which the data needs to be fetched'}}, 'required': ['github_id']}), type='function')], usage=None, temperature=1.0)

In [48]:
import json
# Wait until the job has finished completed
# (you can re-run this step to check the completed_at state)
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)

def get_outputs_for_tool_call(tool_call):
    print("A-1.1", tool_call)
    github_id = json.loads(tool_call.function.arguments)['github_id']
    profile_data = get_github_profile(github_id)
    return {
        "tool_call_id": tool_call.id,
        "output": profile_data
    }

if(run.required_action):
    print("A-0")
    print(run.required_action)
    tool_calls = run.required_action.submit_tool_outputs.tool_calls
    tool_outputs = [get_outputs_for_tool_call(tool_call) for tool_call in tool_calls]
    print("A-1")
    print(tool_outputs)

    run = client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=tool_outputs
    )
    print("A-2")
    print(run)
else:
    print("B")
    print(run)



A-0
RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_2wYYdPrSlQrjJIF42YE1nNx2', function=Function(arguments='{\n  "github_id": "ethansnow2012"\n}', name='get_github_data'), type='function')]), type='submit_tool_outputs')
A-1.1 RequiredActionFunctionToolCall(id='call_2wYYdPrSlQrjJIF42YE1nNx2', function=Function(arguments='{\n  "github_id": "ethansnow2012"\n}', name='get_github_data'), type='function')
Getting github profile for user with id:  ethansnow2012
A-1
[{'tool_call_id': 'call_2wYYdPrSlQrjJIF42YE1nNx2', 'output': 'I am EthanKao. Repo count: 53. Id: ethansnow2012.'}]
A-2
Run(id='run_s5xLDsyxoLSJY8ALRdsDhZ1w', assistant_id='asst_pc2Su2UHd6IV7i3WwaYAhCqF', cancelled_at=None, completed_at=None, created_at=1712392023, expires_at=1712392623, failed_at=None, file_ids=[], instructions='hello plz tell how many repo i have on github. Id: ethansnow2012', last_error=None, metadata={}, model='gpt-4', object='thread.run', re

In [49]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
run

Run(id='run_s5xLDsyxoLSJY8ALRdsDhZ1w', assistant_id='asst_pc2Su2UHd6IV7i3WwaYAhCqF', cancelled_at=None, completed_at=1712392497, created_at=1712392023, expires_at=None, failed_at=None, file_ids=[], instructions='hello plz tell how many repo i have on github. Id: ethansnow2012', last_error=None, metadata={}, model='gpt-4', object='thread.run', required_action=None, started_at=1712392496, status='completed', thread_id='thread_0UlN4CHI6NpzcrZnRq9Z62Dh', tools=[FunctionTool(function=FunctionDefinition(name='get_github_data', description='Retrieves data from a specific API using a GitHub ID. This function is called only if the data has not been previously retrieved.', parameters={'type': 'object', 'properties': {'github_id': {'type': 'string', 'description': 'The GitHub ID for which the data needs to be fetched'}}, 'required': ['github_id']}), type='function')], usage=Usage(completion_tokens=39, prompt_tokens=369, total_tokens=408), temperature=1.0)

In [50]:
# Show the latest message!
from pprint import pp
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

pp(messages)

SyncCursorPage[Message](data=[Message(id='msg_yG1jAQOS9bRi5AW1tCHf3sKl', assistant_id='asst_pc2Su2UHd6IV7i3WwaYAhCqF', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='According to the retrieved data, you have 53 repositories on GitHub.'), type='text')], created_at=1712392497, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_s5xLDsyxoLSJY8ALRdsDhZ1w', status=None, thread_id='thread_0UlN4CHI6NpzcrZnRq9Z62Dh'), Message(id='msg_pT9dIJLgRDvtXFkxMHHAJp4n', assistant_id='asst_pc2Su2UHd6IV7i3WwaYAhCqF', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="I'm sorry, but as an artificial intelligence model, I don't have access to your Github data or any other personal records unless they are shared with me during our conversation. You should be able to check the number of repositories you have on your Github profile page or use GitHub's API."), type='text')], creat

SyncCursorPage[Message](data=[Message(id='msg_yG1jAQOS9bRi5AW1tCHf3sKl', assistant_id='asst_pc2Su2UHd6IV7i3WwaYAhCqF', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='According to the retrieved data, you have 53 repositories on GitHub.'), type='text')], created_at=1712392497, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_s5xLDsyxoLSJY8ALRdsDhZ1w', status=None, thread_id='thread_0UlN4CHI6NpzcrZnRq9Z62Dh'), Message(id='msg_pT9dIJLgRDvtXFkxMHHAJp4n', assistant_id='asst_pc2Su2UHd6IV7i3WwaYAhCqF', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="I'm sorry, but as an artificial intelligence model, I don't have access to your Github data or any other personal records unless they are shared with me during our conversation. You should be able to check the number of repositories you have on your Github profile page or use GitHub's API."), type='text')], created_at=1712391901, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_MkluyCoH7MEVWO2x0yntYlWM', status=None, thread_id='thread_0UlN4CHI6NpzcrZnRq9Z62Dh')], object='list', first_id='msg_yG1jAQOS9bRi5AW1tCHf3sKl', last_id='msg_pT9dIJLgRDvtXFkxMHHAJp4n', has_more=False)
